# Protein feature extraction pipeline

This notebook will contain the pipeline for extracting features from protein sequences. It will be used as a way to show the output without needing to run the `pipeline.py` file locally.

In [1]:
import pyarrow as pa
import pandas as pd
import os
import glob
import logging
from fondant.pipeline import Pipeline
from fondant.pipeline.runner import DockerRunner

logging.basicConfig(
    level=logging.DEBUG,
    format='%(asctime)s - %(levelname)s - %(message)s',
    datefmt='%Y-%m-%d %H:%M:%S'
)

from config import MOCK_DATA_PATH_FONDANT

# check if the manifest file is removed.
REMOVED_MANIFEST = False

# check if the output folder exists
OUTPUT_FOLDER = None

## Generate Mock data

In [2]:
!python utils/generate_mock_data.py

In [3]:
# show content of the mock data
import pandas as pd
mock_df = pd.read_parquet("." + MOCK_DATA_PATH_FONDANT)  # dot added to make it relative to the current directory
mock_df

,sequence,name
0,MNQRGMPIQSLVTNVKINRLEENDCIHTRHRVRPGRTDGKNLHAMM...,Seq1
1,MAGLKPEVPLHDGINKFGKSDFAGQEGPKIVTTTDKALLVANGALK...,Seq2
2,MVDLKKELKNFVDSDFPGSPKQEAQGIDVRILLSFNNAAFREALII...,Seq3
3,MELILAKARLEFECDWGLLMLEPCVPPTKIFADRNYAVGVMFESDK...,Seq4
4,MRVLCDGSTGYACAKNTRIRFREKVASVLAKIQGYEQTFPHHMPNM...,Seq5


## Loading the dataset

In [4]:
# Create a new pipeline

BASE_PATH = ".fondant"
PIPELINE_NAME = "feature_extraction_pipeline"

pipeline = Pipeline(
	name=PIPELINE_NAME,
	base_path=BASE_PATH,
	description="A pipeline to extract features from protein sequences."
)

## Creating the pipeline

In [5]:
# Read the dataset

dataset = pipeline.read(
	"load_from_parquet",
	arguments={
		"dataset_uri": MOCK_DATA_PATH_FONDANT,
	},
	produces={
		"sequence": pa.string()
	}
)

[2024-07-17 12:10:36,673 | fondant.pipeline.pipeline | INFO] The consumes section of the component spec is not defined. Can not infer consumes of the OperationSpec. Please define a consumes section in the dataset interface. 


## Components

---

### generate_protein_sequence_checksum_component

This component generates a checksum for the protein sequence.

---

### biopython_component

Extracts features from the protein sequence using Biopython.

---

### iFeatureOmega_component

Extracts features from the protein sequence using the [iFeatureOmega-CLI GitHub repo](https://github.com/Superzchen/iFeatureOmega-CLI). Arguments are used to specify the type of features to extract.

---

### filter_pdb_component

Filters PDB files that are already predicted to avoid redundant predictions. Arguments need to be specified before running the pipeline:
```json
"storage_type": "local",
"pdb_path": "/data/<your-pdb-folder-path>",
"bucket_name": "your-bucket-name",
"project_id": "your-project-id",
"google_cloud_credentials_path": "/data/<your-credentials>.json"
```

If only using local, keep bucket_name, project_id, and google_cloud_credentials_path as empty strings. Using remote requires a Google Cloud Storage bucket with credentials and a project ID.

---

### predict_protein_3D_structure_component

Predicts the 3D structure of the protein using ESMFold. This component requires a `.env` file with the following variables:
```env
HF_API_KEY=""
HF_ENDPOINT_URL=""
```

---

### store_pdb_component

Stores the PDB files in the provided storage_type. Arguments need to be specified before running the pipeline:
```json
"storage_type": "local",
"pdb_path": "/data/<your-pdb-folder-path>",
"bucket_name": "your-bucket-name",
"project_id": "your-project-id",
"google_cloud_credentials_path": "/data/<your-credentials>.json"
```

If only using local, keep bucket_name, project_id, and google_cloud_credentials_path as empty strings. Using remote requires a Google Cloud Storage bucket with credentials and a project ID.

---

### msa_component

Generates the multiple sequence alignment for the protein sequence using [Clustal Omega](http://www.clustal.org/omega/). It's recommended to use a smaller number of sequences or none at all due to potential time consumption.

---

### unikp_component

Uses the UniKP endpoint on HuggingFace to predict the kinetic parameters of a protein sequence and substrate (SMILES) combination. See README for the description of the contents of this file.

```yaml
"protein_smiles_path": "/data/<path_protein_smiles>"
```

---

### peptide_component

Calculates the features from the protein sequence using the `peptides` package.

---

### deepTMpred_component

Predicts the transmembrane regions of the protein sequence using the [DeepTMpred GitHub repository](https://github.com/ISYSLAB-HUST/DeepTMpred)

In [6]:
_ = dataset.apply(
    "./components/biopython_component"
).apply(
    "./components/generate_protein_sequence_checksum_component"
).apply(
    "./components/iFeatureOmega_component",
    # currently forcing the number of rows to 5, but there needs to be a better way to do this, see readme for more info
    input_partition_rows=5,
    arguments={
        "descriptors": ["AAC", "CTDC", "CTDT"]
    }
).apply(
    "./components/filter_pdb_component",
    arguments={
        "method": "local",
        "local_pdb_path": "/data/pdb_files",
        "bucket_name": "",
        "project_id": "",
        "google_cloud_credentials_path": ""
    }
).apply(
    "./components/predict_protein_3D_structure_component",
).apply(
    "./components/store_pdb_component",
    arguments={
        "method": "local",
        "local_pdb_path": "/data/pdb_files/",
        "bucket_name": "elated-chassis-400207_dbtl_pipeline_outputs",
        "project_id": "elated-chassis-400207",
        "google_cloud_credentials_path": "/data/google_cloud_credentials.json"
    }
).apply(
    "./components/msa_component",
    input_partition_rows='10000'
).apply(
    "./components/pdb_features_component"
# ).apply(
#     "./components/unikp_component",
#     arguments={
#         "target_molecule_smiles": "/data/target_molecule_smiles.json",
#     },
).apply(
    "./components/peptide_features_component"
# ).apply(
#     "./components/DeepTMpred_component"
)

[2024-07-17 12:10:36,687 | fondant.pipeline.pipeline | WARNING] Component `Biopython component` has an image tag set to latest. Caching for the component will be disabled to prevent unpredictable behavior due to images updates
[2024-07-17 12:10:36,691 | fondant.pipeline.pipeline | WARNING] Component `Generate Protein Sequence Checksum Component` has an image tag set to latest. Caching for the component will be disabled to prevent unpredictable behavior due to images updates


InvalidComponentSpec: {'type': 'map_'} is not valid under any of the given schemas

Failed validating 'anyOf' in schema['properties']['produces']['additionalProperties']:
    {'anyOf': [{'$ref': '#/definitions/field'},
               {'$ref': '#/definitions/additionalProperties'}]}

On instance['produces']['unikp_kinetic_prediction']:
    {'type': 'map_'}

## Run the pipeline

The `pipeline.py` file needs to be run using the command line. The following command will run the pipeline:

```bash
fondant < full_path_to_pipeline.py >\data:/data
```

In [ ]:
# import shutil

# remove the most recent output folder if the manifest file is removed
# without a manifest file in the most recent output folder, the pipeline cannot be run
# if OUTPUT_FOLDER and REMOVED_MANIFEST:
# 	shutil.rmtree(OUTPUT_FOLDER)
# 	# remove cache
# 	shutil.rmtree(os.path.join(BASE_PATH, PIPELINE_NAME, "cache"))

# get current full path to the project
mounted_data = os.path.join(os.path.abspath("data"), ":/data")

DockerRunner().run(input=pipeline, extra_volumes=mounted_data)

## Results

The following results have been taken from the output of the pipeline, which is stored in the `.fondant` directory. This directory contains the output of each component, together with the cache of the previous run. Currently, the pipeline doesn't implement the `write_to_file` component, so the results will be taken individually from the output of each component.

In [ ]:
# find the most recent output folder
# get the most recent folder in the folder named: BASE_PATH + PIPELINE_NAME + PIPELINE_NAME-<timestamp>
matching_folders = glob.glob(f"{BASE_PATH}/{PIPELINE_NAME}/{PIPELINE_NAME}-*")

if matching_folders:
    last_folder = max(matching_folders, key=os.path.getctime)

logging.info(f"Last folder: {last_folder}")


In [ ]:
from pathlib import Path

def merge_parquet_folders(folder_path):
    df_list = []

    for folder in Path(folder_path).iterdir():
        if folder.is_dir():
            logging.info(f"Reading parquet partitions from: {folder}")
            parquet_files = list(folder.glob("*.parquet"))
            logging.info(f"Found {len(parquet_files)} parquet files")
            dfs = [pd.read_parquet(file) for file in parquet_files]
            dfs = [x for x in dfs if not x.empty]
            if len(dfs) == 0:
                continue
            df = pd.concat(dfs)
            df_list.append(df)

    return df_list

In [ ]:
dataframe_list = merge_parquet_folders(last_folder)


df_final = pd.concat(dataframe_list, axis=1)
df_final = df_final.loc[:,~df_final.columns.duplicated()]




In [ ]:
df_final[["sequence", "pdb_string", "msa_sequence"]].to_json("test_json.json", orient="records")

In [ ]:
# filtering out columns that are not properly stored in a csv
columns_to_remove = ['pdb_string']
df_final = df_final.drop(columns=columns_to_remove)

# write to file
df_final.to_csv(f"{last_folder}/final_output.csv", index=False)